Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [20]:
from random import choices
import numpy as np
import numpy.random as rand
from copy import deepcopy

import lab9_lib

In [14]:
INDIVIDUAL_SIZE = 1000

In [2]:
fitness = lab9_lib.make_problem(1)
for n in range(2**10):
    ind = [int(b) for b in str('{0:010b}'.format(n))]
    if fitness(ind) == 1 :
        print(ind)

print(fitness.calls)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1024 0


In [40]:
class Individual :
    def __init__(self, fitness_func, genotype = None) -> None:
        if genotype == None :
            self.genotype = [rand.choice([0,1]) for _ in range(INDIVIDUAL_SIZE)]
        else :
            self.genotype = genotype
        self.fitness = fitness_func(self.genotype)

    # Overload [] method
    def __getitem__(self, key) :
        return self.genotype[key]
    
    # Overload [] assignment method
    def __setitem__(self, key, newvalue) :
        self.genotype[key] = newvalue

In [50]:
class Population :
    def __init__(self, size, offspring_size, fitness) -> None:
        self.size = size
        self.offsrping_size = offspring_size
        self.individuals = [Individual(fitness) for _ in range(self.size)]
        self.fitness = fitness

    # Allows to iterate directly over individuals
    def __iter__(self) :
        return self.individuals.__iter__()

    # Mutates one random gene of an individual
    def mutate(self, individual : Individual) :
        # Mutation
        new_ind = deepcopy(individual)
        loci = rand.randint(0, INDIVIDUAL_SIZE)
        new_ind[loci] = not new_ind[loci]
        
        # Fitness update
        new_ind.fitness = self.fitness(new_ind.genotype)
        
        return new_ind
    
    # Recombines two parents using uniform crossover
    def crossover(self, ind1 : Individual, ind2 : Individual) :
        new_ind = Individual(self.fitness, [rand.choice([ind1[i], ind2[i]]) for i in range(INDIVIDUAL_SIZE)])
        return new_ind

    # Perform parent selection using tournament selection
    def parent_selection(self, parent_number, tournament_size) :
        parents = []
        for _ in range(parent_number) :
            candidates = rand.choice(self.individuals, size=tournament_size)
            parent = max(candidates, key= lambda i : i.fitness)
            parents.append(parent)

        return parents
    
    # Create offspring_size new individuals, either by mutation or by crossover
    def create_offspring(self, parents, mutation_prob) :
        new_individuals = []
        for _ in range(self.offsrping_size) :
            if rand.rand() < mutation_prob :
                parent = rand.choice(parents)
                new_individuals.append(self.mutate(parent))
            else :
                par1, par2 = rand.choice(parents, size=2)
                new_individuals.append(self.crossover(par1, par2))
        
        return new_individuals
    
    # Determine the survivors of the population based on their fitness
    def survivor_selection(self, offspring) :
        total_population = self.individuals
        total_population.extend(offspring)
        # Sort all individuals by descending fitness, and keep the fittest one to conserve population size
        surivors = sorted(total_population, key = lambda i : i.fitness, reverse=True)[:self.size]
        return surivors

    # Complete process of a generation
    def evolve(self, parent_number = None, tournament_size = None, mutation_prob = 0.1) :
        if parent_number == None :
            parent_number = int(np.ceil(self.size / 5))
        if tournament_size == None :
            tournament_size = int(np.ceil(parent_number / 2))
        parents = self.parent_selection(parent_number, tournament_size)
        offspring = self.create_offspring(parents, mutation_prob)
        self.individuals = self.survivor_selection(offspring)


In [55]:
test = Population(10,50, fitness)
print(np.mean([ind.fitness for ind in test]))
print('----------')
for _ in range(100) :
    test.evolve()
print(np.mean([ind.fitness for ind in test]))

0.49420000000000003
----------
0.623


In [56]:
fitness.calls

6764